# With ChatModel

In [23]:
import json

from datetime import datetime

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser

In [24]:
def save_result(output, prompt_type, model, init, best_practice):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    result = {}
    result['model'] = model
    result['creation_timestamp'] = date + ' ' + time
    result['prompts'] = {
        'init_uri': init,
        'best_practice_uri': best_practice}
    result['output'] = output

    with open('results/'+prompt_type+'/output_'+date+'_'+time+'.json', 'w') as f:
        json.dump(result, f, indent=4)

In [25]:
model_name = 'gpt-4'

model = ChatOpenAI(
    model=model_name,
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

In [26]:
with open('./data/json/last_row.json') as f:
    json_sample = json.load(f)

## Summary Length

In [27]:
init_prompt_name = 'initPrompt_V1.1.1.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'summaryLengthPrompt_V1.0.2.json'
bestPractice_prompt = load_prompt('prompts/summary/'+bestPractice_prompt_name).format(min= 6, max= 10)

In [28]:
chain = init_prompt | model | StrOutputParser()
print(init_prompt.format(role="Software Engineer", best_practice=bestPractice_prompt, ticket=json_sample))

 ### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Working with these issue trackers, you must consider multiple text-based best practices to increase understandability and reduce time.
Revise a ticket for compliance with these best practices and offer recommendations if needed.
The best practice to check is: 

Best Practice: Summary Length
Your Task is to check if the ticket summary is between 6 and 10 words.
If the word count is in this range, return the original summary.
Otherwise, recommend a new summary for the ticket.
Please return only the summary string and nothing else.


### Context ###
A ticket consists of multiple fields. You are provided with the fields and a short description for each field.
Assignee: The person responsible to resolve the issue.
Comments: Community discussion on the issue, including author, timestamp and content.
Components: Project components to which the issue belongs.
CreatedDate: The tim

In [29]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

'"Sourcetree crashes upon opening repeatedly"'

In [30]:
save_result(output, 'summary', model_name, init_prompt_name, bestPractice_prompt_name)


In [31]:
# # maybe usefull for later
# def run(model_name, init_prompt_name, sumLen_prompt_name):
#     model = ChatOpenAI(
#         model=model_name,
#         api_key=open('api.txt', 'r').read(),
#         temperature=0
#     )
#     with open('./data/json/last_row.json') as f:
#         json_sample = json.load(f)

#     init_prompt = load_prompt('prompts/init/'+init_prompt_name)
#     sumLen_prompt = load_prompt('prompts/summary/'+sumLen_prompt_name).format(min= 6, max= 10)
    
#     chain = init_prompt | model | StrOutputParser()

#     output = chain.invoke({"best_practice": sumLen_prompt,"ticket": json_sample})

#     save_result(output, 'summary', model_name, init_prompt_name, sumLen_prompt_name)  

# run('gpt-4', 'initPrompt_V1.0.0.json', 'summaryLengthPrompt_V1.0.1.json')  

## Description Length

In [32]:
init_prompt_name = 'initPrompt_V1.1.1.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'descriptionLengthPrompt_V1.0.0.json'
bestPractice_prompt = load_prompt('prompts/description/'+bestPractice_prompt_name).format(min= 29, max= 111)

chain = init_prompt | model | StrOutputParser()

In [33]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

"'Source tree crashes every time I open it. I have uninstalled and reinstalled the software multiple times but the issue persists. This issue is occurring on version 3.4.4. I have tried to update to the latest version but our software portal only has version 3.2.6 available. Any assistance in resolving this issue would be greatly appreciated.'"

In [34]:
save_result(output, 'description', model_name, init_prompt_name, bestPractice_prompt_name)

## Field Update

In [35]:
init_prompt_name = 'initPrompt_V1.1.1.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'updatePrompt_V1.2.0.json'
bestPractice_prompt = load_prompt('prompts/update/'+bestPractice_prompt_name).format()

chain = init_prompt | model | StrOutputParser()

In [36]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

"Changes Made:\n1. Updated 'Assignee' field from None to 'Shefali Bhandary' as she was the one who reported the issue and also participated in the comments to resolve it.\n2. Updated 'TimeSpent' field from None to '2 months 5 days' calculated from 'CreatedDate' and 'ResolvedDate'.\n3. Updated 'VersionsFixed' field from None to '3.4.5' based on the last comment by 'Oleksandr Naumenko'.\n4. Added 'Software Update' to 'Labels' field as the issue was resolved by updating the software.\n\nRecommendations:\n1. Always assign an issue to a team member to ensure accountability.\n2. Keep track of the time spent on resolving the issue for better project management.\n3. Update the 'VersionsFixed' field once the issue is resolved in a particular version.\n4. Use 'Labels' field to categorize the issue for easy tracking and reporting."

In [37]:
save_result(output, 'update', model_name, init_prompt_name, bestPractice_prompt_name)

## Toxic Speech Detection

In [38]:
init_prompt_name = 'initPrompt_V1.1.1.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'toxicSpeechPrompt_V1.0.1.json'
bestPractice_prompt = load_prompt('prompts/toxicSpeech/'+bestPractice_prompt_name).format()

chain = init_prompt | model | StrOutputParser()

In [39]:
output = chain.invoke({"role":"Content Moderator", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

"After reviewing the comments in the ticket, it is clear that all the comments are professional and related to the issue at hand. There is no toxic speech detected in the comments. The authors of the comments are 'Vipin Yadav', 'Shefali Bhandary', and 'Oleksandr Naumenko'. All of them have maintained a professional tone while discussing the issue. \n\nTherefore, 'No toxic speech detected.'"

In [40]:
save_result(output, 'toxicSpeech', model_name, init_prompt_name, bestPractice_prompt_name)